# Evolver Loop 6 Analysis: Fixing Overlap Detection

## Critical Issue
exp_005 failed with "Overlapping trees in group 126"

Our relate() method found N=60 but missed N=126. We need:
1. More robust overlap detection
2. OR just use baseline for ALL N values that differ from baseline

In [1]:
import sys
sys.path.insert(0, '/home/code')

import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree

# Load submissions
baseline_path = '/home/code/experiments/000_baseline/submission.csv'
failed_path = '/home/code/experiments/005_fixed_submission/submission.csv'

baseline_df = pd.read_csv(baseline_path)
failed_df = pd.read_csv(failed_path)

print(f"Baseline shape: {baseline_df.shape}")
print(f"Failed submission shape: {failed_df.shape}")

Baseline shape: (20100, 4)
Failed submission shape: (20100, 4)


In [2]:
# Define the Christmas tree polygon
def get_tree_polygon(x, y, angle):
    """Create a Christmas tree polygon at (x, y) with given rotation angle."""
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    tier_1_y = 0.5
    tier_2_y = 0.25
    base_y = 0.0
    trunk_bottom_y = -trunk_h
    
    vertices = [
        (0.0, tip_y),
        (top_w / 2, tier_1_y),
        (top_w / 4, tier_1_y),
        (mid_w / 2, tier_2_y),
        (mid_w / 4, tier_2_y),
        (base_w / 2, base_y),
        (trunk_w / 2, base_y),
        (trunk_w / 2, trunk_bottom_y),
        (-trunk_w / 2, trunk_bottom_y),
        (-trunk_w / 2, base_y),
        (-base_w / 2, base_y),
        (-mid_w / 4, tier_2_y),
        (-mid_w / 2, tier_2_y),
        (-top_w / 4, tier_1_y),
        (-top_w / 2, tier_1_y),
    ]
    
    poly = Polygon(vertices)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, xoff=x, yoff=y)
    return poly

def get_trees_for_n(df, n):
    """Extract trees for a specific N value."""
    prefix = f"{n:03d}_"
    n_data = df[df['id'].str.startswith(prefix)].copy()
    
    trees = []
    for _, row in n_data.iterrows():
        x = float(str(row['x']).lstrip('s'))
        y = float(str(row['y']).lstrip('s'))
        angle = float(str(row['deg']).lstrip('s'))
        trees.append((x, y, angle))
    return trees

print("Functions defined")

Functions defined


In [3]:
# Check N=126 specifically - this is where Kaggle found the overlap
n = 126
failed_trees = get_trees_for_n(failed_df, n)
baseline_trees = get_trees_for_n(baseline_df, n)

print(f"N={n}: Failed has {len(failed_trees)} trees, Baseline has {len(baseline_trees)} trees")

# Check if they're the same
same = True
for i, (f, b) in enumerate(zip(failed_trees, baseline_trees)):
    if f != b:
        same = False
        print(f"Tree {i} differs:")
        print(f"  Failed: {f}")
        print(f"  Baseline: {b}")
        break

print(f"\nN={n} same in both? {same}")

N=126: Failed has 126 trees, Baseline has 126 trees
Tree 0 differs:
  Failed: (-1.185081633737004, 2.0442908326874347, 343.3181854858815)
  Baseline: (-1.18506451723532, 2.044284600228772, 343.3181389566377)

N=126 same in both? False


In [4]:
# Check for overlaps in N=126 using multiple methods
def check_overlaps_area(trees, tolerance=0):
    """Check for overlaps using intersection area."""
    polygons = [get_tree_polygon(x, y, a) for x, y, a in trees]
    
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                area = intersection.area
                if area > tolerance:
                    overlaps.append((i, j, area))
    return overlaps

def check_overlaps_relate(trees):
    """Check for overlaps using relate() - more accurate."""
    polygons = [get_tree_polygon(x, y, a) for x, y, a in trees]
    
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            relate = polygons[i].relate(polygons[j])
            # relate[0] == '2' means 2D interior intersection
            if relate[0] == '2':
                intersection = polygons[i].intersection(polygons[j])
                overlaps.append((i, j, intersection.area, relate))
    return overlaps

def check_overlaps_buffer(trees, buffer_size=1e-10):
    """Check for overlaps using buffer method."""
    polygons = [get_tree_polygon(x, y, a) for x, y, a in trees]
    
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            # Shrink polygons slightly and check intersection
            p1 = polygons[i].buffer(-buffer_size)
            p2 = polygons[j].buffer(-buffer_size)
            if p1.intersects(p2) and not p1.touches(p2):
                overlaps.append((i, j, p1.intersection(p2).area))
    return overlaps

print("Checking N=126 for overlaps...")
print(f"\nArea method (tol=0): {len(check_overlaps_area(failed_trees, 0))} overlaps")
print(f"Relate method: {len(check_overlaps_relate(failed_trees))} overlaps")
print(f"Buffer method: {len(check_overlaps_buffer(failed_trees))} overlaps")

# Show details
relate_overlaps = check_overlaps_relate(failed_trees)
if relate_overlaps:
    print(f"\nRelate overlaps:")
    for o in relate_overlaps[:5]:
        print(f"  Trees {o[0]}-{o[1]}: area={o[2]:.2e}, relate={o[3]}")

Checking N=126 for overlaps...

Area method (tol=0): 0 overlaps
Relate method: 0 overlaps


Buffer method: 0 overlaps


In [5]:
# Check ALL N values in the failed submission for overlaps using relate()
print("Checking ALL N values for overlaps using relate() method...")

problematic_ns = []
for n in range(1, 201):
    trees = get_trees_for_n(failed_df, n)
    overlaps = check_overlaps_relate(trees)
    if overlaps:
        problematic_ns.append((n, overlaps))
        print(f"N={n}: {len(overlaps)} overlaps")
        for o in overlaps[:3]:
            print(f"  Trees {o[0]}-{o[1]}: area={o[2]:.2e}, relate={o[3]}")

print(f"\nTotal problematic N values: {len(problematic_ns)}")

Checking ALL N values for overlaps using relate() method...



Total problematic N values: 0


In [6]:
# Check which N values differ from baseline
print("Checking which N values differ from baseline...")

differing_ns = []
for n in range(1, 201):
    failed_trees = get_trees_for_n(failed_df, n)
    baseline_trees = get_trees_for_n(baseline_df, n)
    
    same = True
    for f, b in zip(failed_trees, baseline_trees):
        if f != b:
            same = False
            break
    
    if not same:
        differing_ns.append(n)

print(f"N values that differ from baseline: {len(differing_ns)}")
print(f"First 20: {differing_ns[:20]}")

Checking which N values differ from baseline...


N values that differ from baseline: 197
First 20: [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]


In [7]:
# SAFEST APPROACH: Use baseline for ALL N values
# The baseline passes Kaggle validation, so it's guaranteed to work
# We can only improve by finding configurations that:
# 1. Are better than baseline
# 2. Pass Kaggle's strict validation

# For now, let's just submit the baseline to confirm it still works
print("Creating a submission using ONLY baseline configurations...")

import shutil
shutil.copy(baseline_path, '/home/submission/submission.csv')

print("Copied baseline to /home/submission/submission.csv")
print("This should pass Kaggle validation with score 70.676102")

Creating a submission using ONLY baseline configurations...
Copied baseline to /home/submission/submission.csv
This should pass Kaggle validation with score 70.676102


In [8]:
# Alternative: Create a TRULY safe ensemble
# Only use non-baseline configurations if they:
# 1. Have a better score
# 2. Pass ALL overlap checks (area, relate, buffer)

def is_config_safe(trees):
    """Check if a configuration is safe using multiple methods."""
    # Method 1: Area check
    area_overlaps = check_overlaps_area(trees, tolerance=0)
    if area_overlaps:
        return False, "area"
    
    # Method 2: Relate check
    relate_overlaps = check_overlaps_relate(trees)
    if relate_overlaps:
        return False, "relate"
    
    # Method 3: Buffer check
    buffer_overlaps = check_overlaps_buffer(trees, buffer_size=1e-12)
    if buffer_overlaps:
        return False, "buffer"
    
    return True, "safe"

print("Checking which differing N values are actually safe...")

safe_improvements = []
for n in differing_ns:
    failed_trees = get_trees_for_n(failed_df, n)
    is_safe, method = is_config_safe(failed_trees)
    
    if is_safe:
        # Calculate scores
        failed_polys = [get_tree_polygon(x, y, a) for x, y, a in failed_trees]
        baseline_trees = get_trees_for_n(baseline_df, n)
        baseline_polys = [get_tree_polygon(x, y, a) for x, y, a in baseline_trees]
        
        def get_score(polys, n):
            all_coords = []
            for p in polys:
                all_coords.extend(list(p.exterior.coords))
            all_coords = np.array(all_coords)
            x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
            y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
            side = max(x_range, y_range)
            return (side ** 2) / n
        
        failed_score = get_score(failed_polys, n)
        baseline_score = get_score(baseline_polys, n)
        
        if failed_score < baseline_score:
            improvement = baseline_score - failed_score
            safe_improvements.append((n, improvement, failed_score, baseline_score))
            print(f"N={n}: SAFE improvement of {improvement:.6f}")
    else:
        print(f"N={n}: UNSAFE ({method})")

print(f"\nTotal safe improvements: {len(safe_improvements)}")

Checking which differing N values are actually safe...
N=2: SAFE improvement of 0.000000
N=5: SAFE improvement of 0.000000
N=7: SAFE improvement of 0.000000
N=8: SAFE improvement of 0.000000
N=9: SAFE improvement of 0.000000
N=11: SAFE improvement of 0.000812
N=12: SAFE improvement of 0.000000
N=13: SAFE improvement of 0.000029
N=14: SAFE improvement of 0.001026
N=15: SAFE improvement of 0.002254
N=16: SAFE improvement of 0.000000


N=18: SAFE improvement of 0.000000
N=21: SAFE improvement of 0.000000
N=22: SAFE improvement of 0.000000
N=23: SAFE improvement of 0.000001
N=24: SAFE improvement of 0.000000


N=25: SAFE improvement of 0.000000


N=32: SAFE improvement of 0.000208
N=33: SAFE improvement of 0.000000
N=34: SAFE improvement of 0.000005
N=35: SAFE improvement of 0.000798


N=36: SAFE improvement of 0.000771
N=37: SAFE improvement of 0.000000
N=38: SAFE improvement of 0.000388


...

N=196: SAFE improvement of 0.000038


N=197: SAFE improvement of 0.000244


N=198: SAFE improvement of 0.000078


N=199: SAFE improvement of 0.000164


N=200: SAFE improvement of 0.000182

Total safe improvements: 170


In [9]:
# Create a truly safe submission using only verified safe improvements
print("\nCreating truly safe submission...")

safe_df = baseline_df.copy()

if safe_improvements:
    for n, improvement, _, _ in safe_improvements:
        # Remove baseline N data
        prefix = f"{n:03d}_"
        safe_df = safe_df[~safe_df['id'].str.startswith(prefix)]
        
        # Add failed N data (which is safe and better)
        failed_n_data = failed_df[failed_df['id'].str.startswith(prefix)]
        safe_df = pd.concat([safe_df, failed_n_data], ignore_index=True)

# Sort by id
safe_df['n'] = safe_df['id'].apply(lambda x: int(x.split('_')[0]))
safe_df['tree_idx'] = safe_df['id'].apply(lambda x: int(x.split('_')[1]))
safe_df = safe_df.sort_values(['n', 'tree_idx']).drop(columns=['n', 'tree_idx'])

print(f"Safe submission shape: {safe_df.shape}")


Creating truly safe submission...


Safe submission shape: (20100, 4)


In [10]:
# Calculate and verify the safe submission score
from utils import score_submission

safe_score, scores_by_n, overlapping_ns = score_submission(safe_df, check_overlaps=True)
baseline_score_check, _, _ = score_submission(baseline_df, check_overlaps=False)

print(f"Safe submission score: {safe_score:.6f}")
print(f"Baseline score: {baseline_score_check:.6f}")
print(f"Improvement over baseline: {baseline_score_check - safe_score:.6f}")
print(f"Overlapping N values (utils check): {overlapping_ns}")
print(f"\nTarget: 68.888293")
print(f"Gap to target: {safe_score - 68.888293:.6f}")

Safe submission score: 70.615788
Baseline score: 70.676102
Improvement over baseline: 0.060314
Overlapping N values (utils check): []

Target: 68.888293
Gap to target: 1.727495


In [11]:
# Final verification using all three overlap methods
print("\nFinal verification of safe submission...")

final_problems = []
for n in range(1, 201):
    trees = get_trees_for_n(safe_df, n)
    is_safe, method = is_config_safe(trees)
    if not is_safe:
        final_problems.append((n, method))
        print(f"N={n}: PROBLEM ({method})")

if not final_problems:
    print("✅ All N values pass all overlap checks!")
else:
    print(f"❌ {len(final_problems)} problematic N values")


Final verification of safe submission...
N=4: PROBLEM (relate)
N=17: PROBLEM (area)


N=30: PROBLEM (relate)


N=40: PROBLEM (area)
N=42: PROBLEM (area)


N=48: PROBLEM (area)


N=53: PROBLEM (area)


...

N=154: PROBLEM (area)


N=161: PROBLEM (area)


N=164: PROBLEM (area)


N=166: PROBLEM (area)


❌ 14 problematic N values


In [12]:
# Fix the problematic N values by replacing with baseline
print("Fixing problematic N values...")

truly_safe_df = safe_df.copy()

for n, method in final_problems:
    print(f"Replacing N={n} (failed {method}) with baseline...")
    
    # Remove problematic N data
    prefix = f"{n:03d}_"
    truly_safe_df = truly_safe_df[~truly_safe_df['id'].str.startswith(prefix)]
    
    # Add baseline N data
    baseline_n_data = baseline_df[baseline_df['id'].str.startswith(prefix)]
    truly_safe_df = pd.concat([truly_safe_df, baseline_n_data], ignore_index=True)

# Sort by id
truly_safe_df['n'] = truly_safe_df['id'].apply(lambda x: int(x.split('_')[0]))
truly_safe_df['tree_idx'] = truly_safe_df['id'].apply(lambda x: int(x.split('_')[1]))
truly_safe_df = truly_safe_df.sort_values(['n', 'tree_idx']).drop(columns=['n', 'tree_idx'])

print(f"\nTruly safe submission shape: {truly_safe_df.shape}")

Fixing problematic N values...
Replacing N=4 (failed relate) with baseline...
Replacing N=17 (failed area) with baseline...
Replacing N=30 (failed relate) with baseline...
Replacing N=40 (failed area) with baseline...
Replacing N=42 (failed area) with baseline...
Replacing N=48 (failed area) with baseline...
Replacing N=53 (failed area) with baseline...
Replacing N=99 (failed relate) with baseline...
Replacing N=102 (failed area) with baseline...
Replacing N=103 (failed area) with baseline...
Replacing N=154 (failed area) with baseline...
Replacing N=161 (failed area) with baseline...
Replacing N=164 (failed area) with baseline...
Replacing N=166 (failed area) with baseline...

Truly safe submission shape: (20100, 4)


In [13]:
# Final verification of truly safe submission
print("Final verification of truly safe submission...")

final_problems_v2 = []
for n in range(1, 201):
    trees = get_trees_for_n(truly_safe_df, n)
    is_safe, method = is_config_safe(trees)
    if not is_safe:
        final_problems_v2.append((n, method))
        print(f"N={n}: PROBLEM ({method})")

if not final_problems_v2:
    print("✅ All N values pass all overlap checks!")
else:
    print(f"❌ {len(final_problems_v2)} problematic N values")

# Calculate score
truly_safe_score, _, _ = score_submission(truly_safe_df, check_overlaps=False)
print(f"\nTruly safe submission score: {truly_safe_score:.6f}")
print(f"Baseline score: {baseline_score_check:.6f}")
print(f"Improvement over baseline: {baseline_score_check - truly_safe_score:.6f}")

Final verification of truly safe submission...
N=4: PROBLEM (relate)


N=17: PROBLEM (area)


N=30: PROBLEM (relate)


N=40: PROBLEM (area)
N=42: PROBLEM (area)


N=48: PROBLEM (area)


...

N=161: PROBLEM (area)


N=164: PROBLEM (area)


N=166: PROBLEM (area)


❌ 14 problematic N values



Truly safe submission score: 70.615788
Baseline score: 70.676102
Improvement over baseline: 0.060314


In [14]:
# Save the truly safe submission
import os
import json

work_dir = '/home/code/experiments/006_truly_safe'
os.makedirs(work_dir, exist_ok=True)

truly_safe_df.to_csv(f'{work_dir}/submission.csv', index=False)

metrics = {
    'cv_score': truly_safe_score,
    'baseline_score': baseline_score_check,
    'improvement_over_baseline': baseline_score_check - truly_safe_score,
    'problematic_ns_fixed': [n for n, _ in final_problems],
    'is_valid': len(final_problems_v2) == 0
}

with open(f'{work_dir}/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

# Copy to submission folder
import shutil
shutil.copy(f'{work_dir}/submission.csv', '/home/submission/submission.csv')

print(f"Saved to {work_dir}/submission.csv")
print(f"Copied to /home/submission/submission.csv")
print(f"Metrics: {metrics}")

Saved to /home/code/experiments/006_truly_safe/submission.csv
Copied to /home/submission/submission.csv
Metrics: {'cv_score': 70.61578810307368, 'baseline_score': 70.67610239809187, 'improvement_over_baseline': 0.060314295018187636, 'problematic_ns_fixed': [4, 17, 30, 40, 42, 48, 53, 99, 102, 103, 154, 161, 164, 166], 'is_valid': False}


In [15]:
# Check if baseline itself has overlaps
print("Checking baseline for overlaps...")

baseline_problems = []
for n in range(1, 201):
    trees = get_trees_for_n(baseline_df, n)
    is_safe, method = is_config_safe(trees)
    if not is_safe:
        baseline_problems.append((n, method))
        print(f"N={n}: PROBLEM ({method})")

if not baseline_problems:
    print("✅ Baseline passes all overlap checks!")
else:
    print(f"❌ Baseline has {len(baseline_problems)} problematic N values")

Checking baseline for overlaps...
N=4: PROBLEM (relate)
N=5: PROBLEM (area)
N=12: PROBLEM (area)
N=16: PROBLEM (area)
N=17: PROBLEM (area)
N=21: PROBLEM (area)
N=23: PROBLEM (area)


N=30: PROBLEM (relate)
N=35: PROBLEM (area)


N=40: PROBLEM (area)
N=41: PROBLEM (area)
N=42: PROBLEM (area)
N=43: PROBLEM (area)
N=45: PROBLEM (area)


N=48: PROBLEM (area)
N=49: PROBLEM (area)


N=53: PROBLEM (area)


...

N=173: PROBLEM (area)


N=185: PROBLEM (area)


N=187: PROBLEM (relate)


N=191: PROBLEM (area)


❌ Baseline has 53 problematic N values


In [16]:
# The baseline passes Kaggle validation with score 70.676102\n# But our local checks find 53 \"overlaps\"\n# This means our overlap detection is TOO STRICT\n\n# Let's look at what Kaggle's actual validation might be\n# From the competition description, trees can \"touch\" but not \"overlap\"\n# The relate() method with '2' in position 0 means interior intersection\n# But maybe Kaggle allows very small interior intersections?\n\n# Let's check the actual overlap areas for baseline\nprint(\"Checking baseline overlap areas...\")\n\nfor n, method in baseline_problems[:10]:  # First 10\n    trees = get_trees_for_n(baseline_df, n)\n    polygons = [get_tree_polygon(x, y, a) for x, y, a in trees]\n    \n    for i in range(len(polygons)):\n        for j in range(i+1, len(polygons)):\n            if polygons[i].intersects(polygons[j]):\n                intersection = polygons[i].intersection(polygons[j])\n                area = intersection.area\n                relate = polygons[i].relate(polygons[j])\n                if area > 0 or relate[0] == '2':\n                    print(f\"N={n}: Trees {i}-{j}: area={area:.2e}, relate={relate}\")"

In [17]:
# The baseline passes Kaggle validation with score 70.676102
# But our local checks find 53 "overlaps"
# This means our overlap detection is TOO STRICT

# Let's look at what Kaggle's actual validation might be
# From the competition description, trees can "touch" but not "overlap"
# The relate() method with '2' in position 0 means interior intersection
# But maybe Kaggle allows very small interior intersections?

# Let's check the actual overlap areas for baseline
print("Checking baseline overlap areas...")

for n, method in baseline_problems[:10]:  # First 10
    trees = get_trees_for_n(baseline_df, n)
    polygons = [get_tree_polygon(x, y, a) for x, y, a in trees]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                area = intersection.area
                relate = polygons[i].relate(polygons[j])
                if area > 0 or relate[0] == '2':
                    print(f"N={n}: Trees {i}-{j}: area={area:.2e}, relate={relate}")

Checking baseline overlap areas...
N=4: Trees 1-2: area=0.00e+00, relate=212101212
N=5: Trees 2-3: area=3.42e-31, relate=212101212
N=12: Trees 7-11: area=3.97e-25, relate=212101212
N=16: Trees 6-15: area=6.87e-30, relate=212101212
N=17: Trees 2-15: area=5.75e-30, relate=212101212
N=21: Trees 2-16: area=3.82e-31, relate=212101212
N=23: Trees 0-4: area=6.72e-31, relate=212101212
N=30: Trees 18-19: area=0.00e+00, relate=2F2101212
N=35: Trees 2-24: area=0.00e+00, relate=212101212
N=35: Trees 6-8: area=5.16e-32, relate=212101212
N=40: Trees 10-17: area=4.93e-32, relate=212101212


In [19]:
# Check N=126 in detail - this is where Kaggle found the overlap
n = 126
failed_trees = get_trees_for_n(failed_df, n)
baseline_trees = get_trees_for_n(baseline_df, n)

print(f"=== N={n} Analysis ===")

print("\nFailed submission overlaps:")
failed_polys = [get_tree_polygon(x, y, a) for x, y, a in failed_trees]
for i in range(len(failed_polys)):
    for j in range(i+1, len(failed_polys)):
        if failed_polys[i].intersects(failed_polys[j]):
            intersection = failed_polys[i].intersection(failed_polys[j])
            area = intersection.area
            relate = failed_polys[i].relate(failed_polys[j])
            if area > 1e-30 or relate[0] == '2':
                print(f"  Trees {i}-{j}: area={area:.2e}, relate={relate}")

print("\nBaseline overlaps:")
baseline_polys = [get_tree_polygon(x, y, a) for x, y, a in baseline_trees]
for i in range(len(baseline_polys)):
    for j in range(i+1, len(baseline_polys)):
        if baseline_polys[i].intersects(baseline_polys[j]):
            intersection = baseline_polys[i].intersection(baseline_polys[j])
            area = intersection.area
            relate = baseline_polys[i].relate(baseline_polys[j])
            if area > 1e-30 or relate[0] == '2':
                print(f"  Trees {i}-{j}: area={area:.2e}, relate={relate}")

=== N=126 Analysis ===

Failed submission overlaps:

Baseline overlaps:


In [20]:
# Kaggle's EXACT overlap detection:
# poly.intersects(other) and not poly.touches(other)

# This is different from checking intersection area!
# intersects() returns True if geometries share any point
# touches() returns True if geometries share boundary points but not interior points

# So the check is: "do they share interior points?"

def check_overlaps_kaggle_exact(trees):
    """Check for overlaps using Kaggle's exact method."""
    from decimal import Decimal, getcontext
    getcontext().prec = 25
    scale_factor = Decimal('1e18')
    
    # Create polygons with Kaggle's exact scaling
    polygons = []
    for x, y, angle in trees:
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        
        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        
        rotated = affinity.rotate(initial_polygon, angle, origin=(0, 0))
        poly = affinity.translate(rotated, 
                                  xoff=float(Decimal(str(x)) * scale_factor),
                                  yoff=float(Decimal(str(y)) * scale_factor))
        polygons.append(poly)
    
    # Check for overlaps using Kaggle's exact method
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlaps.append((i, j))
    
    return overlaps

# Test on N=126
print("Testing N=126 with Kaggle's exact method...")
failed_overlaps = check_overlaps_kaggle_exact(failed_trees)
baseline_overlaps = check_overlaps_kaggle_exact(baseline_trees)

print(f"Failed N=126: {len(failed_overlaps)} overlaps")
print(f"Baseline N=126: {len(baseline_overlaps)} overlaps")

if failed_overlaps:
    print(f"Failed overlaps: {failed_overlaps[:5]}")

Testing N=126 with Kaggle's exact method...
Failed N=126: 1 overlaps
Baseline N=126: 0 overlaps
Failed overlaps: [(42, 86)]


In [ ]:
# Save the safe submission
import os
import json

work_dir = '/home/code/experiments/006_truly_safe'
os.makedirs(work_dir, exist_ok=True)

safe_df.to_csv(f'{work_dir}/submission.csv', index=False)

metrics = {
    'cv_score': safe_score,
    'baseline_score': baseline_score_check,
    'improvement_over_baseline': baseline_score_check - safe_score,
    'safe_improvements_count': len(safe_improvements),
    'is_valid': len(final_problems) == 0
}

with open(f'{work_dir}/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

# Copy to submission folder
import shutil
shutil.copy(f'{work_dir}/submission.csv', '/home/submission/submission.csv')

print(f"Saved to {work_dir}/submission.csv")
print(f"Copied to /home/submission/submission.csv")
print(f"Metrics: {metrics}")